In [10]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 3, in <module>
  File "stdmqttas.py", line 3, in <module>
  File "mqtt_as.py", line 35, in <module>
KeyboardInterrupt: 
MicroPython v1.17-142-g8e1fcd490 on 2021-11-11; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [11]:
import os
print(os.rename("config.txt", "Dconfig.txt"))

None


In [12]:
%ls

Listing directory '/'.
     5506    BNO055_class.py
      350    Dconfig.txt
      139    boot.py
     5830    main.py
    23172    mqtt_as.py
     2679    stdmqttas.py
      969    utils.py


In [14]:
%fetchfile config.txt --print

connection0  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection2  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection3  DoESLiverpool,decafbad00,mosquitto.doesliverpool.xyz
pinled       2
pinscl       13
pinsda       12
wdtsecs      300
devices      SCD30
position     beckasroom

In [35]:
# ESP8266 version
%sendtofile config.txt

connection0  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection1  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection2  DoESLiverpool,decafbad00,10.0.100.1
pinled       2
pinscl       5
pinsda       4
pinpir       13

Sent 7 lines (238 bytes) to config.txt.


In [92]:
# ESP32 version  MH-ET Live
%sendtofile config.txt

connection2  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection0  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection1  DoESLiverpool,decafbad00,10.0.100.1
pinled       2
pinscl       12
pinsda       13
wdtsecs      300
devices      SCD30
position     livingroom

Sent 9 lines (284 bytes) to config.txt.


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [13]:
# ESP32 version Devkit
%sendtofile config.txt

connection0  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection2  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection3  DoESLiverpool,decafbad00,mosquitto.doesliverpool.xyz
pinled       2
pinscl       13
pinsda       12
wdtsecs      300
devices      SCD30
position     beckasroom

Sent 9 lines (301 bytes) to config.txt.


In [97]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py
# devices SCD30,SCD41

Sent 82 lines (2679 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [98]:
%serialconnect
# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

# !!! The redo for different connections isn't working so put the one 
# that is active into connection0

Sent 647 lines (23171 bytes) to mqtt_as.py.


In [84]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [99]:
%sendtofile main.py

import time, sys, machine, struct
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from stdmqttas import shortmac, topicstem, itertools_count
from stdmqttas import pinled, pinledOnvalue, flashpinled
from mqtt_as import MQTTClient

import network
network.WLAN().active(0)  # disable the connection at startup
flashpinled(5, 300, 300)

def crc8(b):
    crc = 0xFF
    for c in b:
        crc ^= c
        for i in range(8):
            crc = 0xFF & (((crc << 1) ^ 0x31) if (crc & 0x80) else (crc << 1))
    return crc

if sys.platform == "esp32":
    i2c = machine.I2C(1, scl=machine.Pin(int(fconfig["pinscl"])), sda=machine.Pin(int(fconfig["pinsda"])))
else:
    i2c = machine.I2C(scl=machine.Pin(int(fconfig["pinscl"])), sda=machine.Pin(int(fconfig["pinsda"])))

topicscd41 = topicstem+"/scd41"
topicscd30 = topicstem+"/scd30"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"
sensorposition = fconfig.get("position", "")
print("topicstem: '%s'" % topicstem)
print("sensorposition: '%s'" % sensorposition)

async def WatchDogtask(client, wdtsecs):
    await asyncio.sleep_ms(20*1000)
    print("Starting Watchdog timer for", wdtsecs, "seconds")
    wd = machine.WDT(timeout=int(wdtsecs*1000))
    while True:
        await asyncio.sleep_ms(1*1000)
        if client.isconnected():
            wd.feed()
            await asyncio.sleep_ms(9*1000)
        elif pinled:
            pinled.value(pinledOnvalue)
            await asyncio.sleep_ms(1*1000)
            pinled.value(not pinledOnvalue)
    

async def SCD30task(client):
    try:
        i2c.writeto(0x61, bytes([0x02, 0x04]))   # soft reset
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x00, 0x10, 0x00, 0x00, crc8([0x00, 0x00])]))   # continuous measurement
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x46, 0x00, 0x00, 0x02, crc8([0x00, 0x02])]))   # 2s measurement interval
        await asyncio.sleep_ms(40)
        for n in itertools_count():
            y = i2c.writeto(0x61, bytes([0x02, 0x02]))  # data ready?
            await asyncio.sleep_ms(4)
            d = i2c.readfrom(0x61, 2)    # ready is 1
            if d == bytes([0x00, 0x01]):
                y = i2c.writeto(0x61, bytes([0x03, 0x00]))  # read data
                await asyncio.sleep_ms(4)
                x = i2c.readfrom(0x61, 18)
                #crc8(x[:2]) == x[2]  etc for every 3 bytes
                co2, temp, humid = struct.unpack(">fff", bytes([x[0],x[1],x[3],x[4], x[6],x[7],x[9],x[10], x[12],x[13],x[15],x[16]]))
                print(n, topicscd30, co2, temp, humid)
                if client.isconnected():
                    await client.publish(topicscd30, "%f %f %f %s"%(co2, temp, humid, sensorposition))
            await asyncio.sleep_ms(500)
    except OSError as e:
        print("SCD30taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(SCD30task(client))

        
async def SCD41task(client):
    try:
        i2c.writeto(0x62, bytes([0x3F, 0x86]))   # stopPeriodicMeasurement
        await asyncio.sleep_ms(500)
        i2c.writeto(0x62, bytes([0x36, 0x82]))   # getSerialNumber
        await asyncio.sleep_ms(1)
        x = i2c.readfrom(0x62, 9)
        s0, s1, s2 = struct.unpack(">HHH", bytes([x[0],x[1], x[3],x[4], x[6],x[7]]))
        print("SCD41 serial number", hex(s0), hex(s1), hex(s2))
        i2c.writeto(0x62, bytes([0x21, 0xB1]))   # startPeriodicMeasurement
        await asyncio.sleep_ms(1)
        for n in itertools_count():
            i2c.writeto(0x62, bytes([0xE4, 0xB8]))   # check ready
            await asyncio.sleep_ms(4)
            d = i2c.readfrom(0x62, 3)
            if not (d[0] == 0x80 and d[1] == 0x00):
                i2c.writeto(0x62, bytes([0xEC, 0x05]))   # readMeasurement
                await asyncio.sleep_ms(1)
                x = i2c.readfrom(0x62, 9)
                co2, utemp, uhumid = struct.unpack(">HHH", bytes([x[0],x[1], x[3],x[4], x[6],x[7]]))
                temp = utemp*175.0/65536.0 - 45.0
                humid = uhumid*100.0/65536.0
                print(n, topicscd41, co2, temp, humid)
                if client.isconnected():
                    await client.publish(topicscd41, "%f %f %f %s"%(co2, temp, humid, sensorposition))
            await asyncio.sleep_ms(500)
    except OSError as e:
        print("SCD41taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(SCD41task(client))
        
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)

print("c1")    
config['subs_cb'] = callbackcmd
print("c2")    
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
print("c4")
client = MQTTClient(config)
print("c5")
client.DEBUG = True

devices = fconfig.get("devices", "SCD30").split(",")
  
aloop = asyncio.get_event_loop()
if "SCD30" in devices:
    aloop.create_task(SCD30task(client))
if "SCD41" in devices:
    aloop.create_task(SCD41task(client))
aloop.create_task(mqttconnecttask(client))
if "wdtsecs" in fconfig:
    aloop.create_task(WatchDogtask(client, float(fconfig["wdtsecs"])))

print("first clearing topicerror")
aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    print("runforever", n)
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))


Sent 149 lines (5830 bytes) to main.py.


In [31]:
%ls

Listing directory '/'.
      139    boot.py
      284    config.txt
     5679    main.py
    23172    mqtt_as.py
     2679    stdmqttas.py
      969    utils.py
